<a href="https://colab.research.google.com/github/amarjeetamrat910/-One-Hot-Encoding-from-Scratch-in-NLP/blob/main/%24Final_Q_A_system_using_the_Gemini_API%2C_LangChain%2C_and_Chroma%2Cipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Install and  Import Libraries *

---



In [ ]:
!pip install --quiet langchain

In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
!pip install pymupdf

import fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 38.9 MB/s eta 0:00:00


In [ ]:
!pip install pymupdf

!pip install --upgrade langchain

!pip install langchain
!pip install -U langchain-community
!pip install langchain_google_genai
!pip install chromadb

!pip install pymupdf
!pip install -U langchain-communit
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
import os
import getpass
from langchain import PromptTemplate
from langchain.docstore.document import Document
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: Could not find a version that satisfies the requirement langchain-communit (from versions: none)
ERROR: No matching distribution found for langchain-communit


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# !pip uninstall fitz # Uninstall the incorrect fitz module
# !pip install pymupdf # Install the correct module for PDF manipulation
# import fitz # Import fitz after installing the correct module

# 2.Environment Setup


---



In [ ]:
def setup_environment(api_key):
    os.environ['GOOGLE_API_KEY'] = api_key

# 3. Document Loading and Processing

---




In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util

## 13 January 2024-------

def semantic_chunking(text, similarity_threshold=0.8):

    sentences = sent_tokenize(text)
    if not sentences:
        return []

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences, convert_to_tensor=True)

    chunks = []
    current_chunk = [sentences[0]]

    for i in range(1, len(sentences)):
        similarity = util.cos_sim(embeddings[i], embeddings[i - 1]).item()
        if similarity >= similarity_threshold:
            current_chunk.append(sentences[i])
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentences[i]]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

if __name__ == "__main__":
    extracted_text = ("Artificial intelligence (AI) is transforming industries. "
                      "Recent advances in machine learning have led to breakthroughs in AI applications. "
                      "The integration of AI into business processes is accelerating. "
                      "At the same time, ethical considerations are becoming increasingly important. "
                      "Organizations must balance innovation with responsibility.")

    semantic_chunks = semantic_chunking(extracted_text, similarity_threshold=0.8)
    for idx, chunk in enumerate(semantic_chunks, 1):
        print(f"Chunk {idx}:\n{chunk}\n")


Chunk 1:
Artificial intelligence (AI) is transforming industries.

Chunk 2:
Recent advances in machine learning have led to breakthroughs in AI applications.

Chunk 3:
The integration of AI into business processes is accelerating.

Chunk 4:
At the same time, ethical considerations are becoming increasingly important.

Chunk 5:
Organizations must balance innovation with responsibility.



In [ ]:
def load_and_process_pdf(file_path, split_start, split_end, chunk_size=500):
    doc = fitz.open(file_path)
    text_content = ""

    for page in doc:
        text_content += page.get_text()

    if split_start in text_content and split_end in text_content:
        text_content_1 = text_content.split(split_start, 1)[1]
        final_text = text_content_1.split(split_end, 1)[0]
    else:
        final_text = text_content

    chunks = semantic_chunking(final_text, similarity_threshold=0.8)

    documents = [
        Document(page_content=chunk, metadata={"source": file_path})
        for chunk in chunks
    ]
    return documents


In [ ]:
'''

def load_and_process_pdf(file_path, split_start, split_end, chunk_size=500):
    doc = fitz.open(file_path)
    text_content = ""

    for page in doc:
        text_content += page.get_text()

    text_content_1 = text_content.split(split_start, 1)
    final_text = text_content_1[0].split(split_end, 1)[0]

    sentences = sent_tokenize(final_text)

    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length <= chunk_size:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    documents = [
        Document(page_content=chunk, metadata={"source": file_path})
        for chunk in chunks
    ]

    return documents

# 4. Vector Store Setup

---




In [ ]:
def setup_vectorstore(docs, model_name, persist_dir):
    gemini_embeddings = GoogleGenerativeAIEmbeddings(model=model_name)
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=gemini_embeddings,
        persist_directory=persist_dir
    )
    return vectorstore

# 5. Retriever Setup

---




In [ ]:
def setup_retriever(vectorstore, k):
    return vectorstore.as_retriever(search_kwargs={"k": k})

# 6. LLM Setup

---




In [ ]:
def setup_llm(llm_model_name, temperature, top_p):
    llm = ChatGoogleGenerativeAI(model=llm_model_name, temperature=temperature, top_p=top_p)
    llm_prompt_template = """
    You are an assistant for question-answering tasks.
    Use the following context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use five sentences maximum and keep the answer concise.\n
    Question: {question} \nContext: {context} \nAnswer:"""
    llm_prompt = PromptTemplate.from_template(llm_prompt_template)
    return llm, llm_prompt

# 7. Combine documents into readable string format




---




In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 8. RAG Chain and Invocation

---





In [ ]:
def create_and_invoke_rag_chain(retriever, llm, llm_prompt, question):
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | llm_prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain.invoke(question)

# 9.Configuration Function for API Integration and Parameters setup

In [ ]:
def configuration():
    return {
        "api_key": getpass.getpass("Enter the API Key"),
        "pdf_path": "/content/BAL Annual Report FY24.pdf",
        "split_start": " audio, image and video.",
        "split_end": "Cloud TPU v5p",
        "model_name": "models/embedding-001",
        "persist_dir": "./chroma_db",
        "k": 3,
        "llm_model_name": "gemini-pro",
        "temperature": 0.7,
        "top_p": 0.85,
        "question": input("Ask your question: ")
    }

# 10.Main function to run the entire process




---



In [ ]:
def main():
  print("Welcome to the Q/A system!","Feel free to ask any questions.","We're here to assist you with your queries.")


  config = configuration()

  setup_environment(config["api_key"])

  docs = load_and_process_pdf(
        config["pdf_path"],
        config["split_start"],
        config["split_end"],
        chunk_size=500
    )

  vectorstore = setup_vectorstore(docs, config["model_name"], config["persist_dir"])

  retriever = setup_retriever(vectorstore, config["k"])

  print(len(retriever.get_relevant_documents("MMLU")))

  llm, llm_prompt = setup_llm(config["llm_model_name"], config["temperature"], config["top_p"])

  answer = create_and_invoke_rag_chain(retriever, llm, llm_prompt, config["question"])

  print(answer)
  print()

if __name__ == "__main__":
    main()

Welcome to the Q/A system! Feel free to ask any questions. We're here to assist you with your queries.
Enter the API Key··········
Ask your question: ceo of bajaj
3
The CEO of Bajaj is Rajiv Bajaj.

